In [2]:
from helper_functions import *

In [4]:
inp = tf.keras.Input(shape=(5))
dense1 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(inp)
dense2 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(dense1)
dense3 = layers.Dense(20, activation=tf.nn.relu, use_bias = True)(dense2)
output = layers.Dense(1, use_bias = True)(dense3)
nn_ctlr=tf.keras.Model(inputs=inp,outputs=output)

In [6]:
onnx_model_path = '../benchmarks/controller_3_20.onnx'
onnx_model = onnx.load(onnx_model_path)

weights_list=[]
i=0
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==1 and i >=0):
        weights_list.append(tensor_array)
    i = i+1
    
bias_list=[]
i=0
for initializer in onnx_model.graph.initializer:
    # Convert the initializer tensor to a NumPy array
    tensor_array = onnx.numpy_helper.to_array(initializer)
    if(i%2==0 and i >0):
        bias_list.append(tensor_array)
    i = i+1
    
for i in range(1, len(weights_list) + 1):
    transposed_weights = np.transpose(weights_list[i-1])
    combined_weights = [transposed_weights, bias_list[i-1]]
    nn_ctlr.layers[i].set_weights(combined_weights)

In [4]:
weights, biases = extract_model_params_tf(nn_ctlr)

In [5]:
biases

[array([ 0.0229921 ,  0.9157833 ,  0.68303037,  0.8829671 ,  0.9406548 ,
        -0.27174762, -0.35092968,  0.5552317 , -0.56850016,  0.91673166,
         0.8542432 , -0.26324683, -0.30767414,  0.3705317 ,  0.43489116,
         0.5938729 ,  0.18588172, -0.80390227,  0.08894817, -0.91160786],
       dtype=float32),
 array([-0.822485  , -0.4742797 , -0.6593712 ,  0.09373701, -0.6307485 ,
        -0.02549562,  0.45734137,  0.93127143,  0.6027336 , -0.8678251 ,
         0.8682161 ,  0.96597815,  0.15609814,  0.9114345 , -0.34187108,
         0.8110454 , -0.9849178 ,  0.00226631, -0.70056045, -0.32336384],
       dtype=float32),
 array([ 0.09043658, -0.50260955,  0.7954128 , -0.84483844,  0.65653545,
        -0.1070924 ,  0.8637377 ,  0.0428166 ,  0.7314259 , -0.30280596,
        -0.2164498 ,  0.27524385,  0.78397036, -0.7493645 , -0.90314686,
        -0.20262143,  0.21611765,  0.87921983, -0.4206973 ,  0.08362798],
       dtype=float32),
 array([-0.279995], dtype=float32)]

In [5]:
# Change property specifications below
layer_no = 4
neuron_no = 1
time_steps = 5
# output_range = [-1.5,0.55]
input_bounds = [[30,30],[1.4,1.4],[30,30.2],[32,32.2],[10,11],[90,110]]

In [6]:
def summon_gurobi(dec, eqn, log, to, focus=0):
    all_enc = dec + eqn
    file_path = "Gurobi_encodings.txt"
    with open(file_path, "w") as file:
        for value in all_enc:
            file.write(str(value) + "\n")
    model=gp.Model("Encodings")
    model.Params.MIPFocus = focus
    model.Params.LogToConsole = log
    model.setParam('TimeLimit', to*60*60)
    model.Params.SolutionLimit = 1

    try:
        f = open(file_path,"r")
        try:
            for l in f:
                exec(l)
        finally:
            f.close()
    except IOError:
        pass

    model.optimize()
    
    return model

In [7]:
def can_go_above(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    
    declare.append(f"A0_1_1 = model.addVar(lb={input_bounds[0][0]}, ub={input_bounds[0][1]}, name='A0_1_1')")
    declare.append(f"A0_2_1 = model.addVar(lb={input_bounds[1][0]}, ub={input_bounds[1][1]}, name='A0_2_1')")
    declare.append(f"c = model.addVar(lb={input_bounds[2][0]}, ub={input_bounds[2][1]}, name='c')")
    declare.append(f"d = model.addVar(lb={input_bounds[3][0]}, ub={input_bounds[3][1]}, name='d')")
    declare.append(f"e = model.addVar(lb={input_bounds[4][0]}, ub={input_bounds[4][1]}, name='e')")

    declare.append(f"A0_3_1 = model.addVar(lb={input_bounds[5][0]}, ub={input_bounds[5][1]}, name='A0_3_1')")
    declare.append(f"A0_4_1 = model.addVar(lb=-2.2, ub=-1.8, name='A0_4_1')")
    declare.append(f"A0_5_1 = model.addVar(lb=-100, ub=-79, name='A0_5_1')")

    equations.append(f"model.addConstr(A0_4_1==c-d)")
    equations.append(f"model.addConstr(A0_5_1==e - A0_3_1)")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")
    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 9999999
    eps=0.0000001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}>={cond[1]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MAXIMIZE)')


    return equations, declare 

In [8]:
def can_go_below(w,b,layer,no,time_steps, cond, input_bounds):
    global equations, declare
    equations=[]
    declare=[]
    inputs=w[0].shape[1]

    
    declare.append(f"A0_1_1 = model.addVar(lb={input_bounds[0][0]}, ub={input_bounds[0][1]}, name='A0_1_1')")
    declare.append(f"A0_2_1 = model.addVar(lb={input_bounds[1][0]}, ub={input_bounds[1][1]}, name='A0_2_1')")
    declare.append(f"c = model.addVar(lb={input_bounds[2][0]}, ub={input_bounds[2][1]}, name='c')")
    declare.append(f"d = model.addVar(lb={input_bounds[3][0]}, ub={input_bounds[3][1]}, name='d')")
    declare.append(f"e = model.addVar(lb={input_bounds[4][0]}, ub={input_bounds[4][1]}, name='e')")

    declare.append(f"A0_3_1 = model.addVar(lb={input_bounds[5][0]}, ub={input_bounds[5][1]}, name='A0_3_1')")
    declare.append(f"A0_4_1 = model.addVar(lb=-2.2, ub=-1.8, name='A0_4_1')")
    declare.append(f"A0_5_1 = model.addVar(lb=-100, ub=-79, name='A0_5_1')")

    equations.append(f"model.addConstr(A0_4_1==c-d)")
    equations.append(f"model.addConstr(A0_5_1==e - A0_3_1)")

    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            declare.append(f"P{i}_{j}_0 = model.addVar(name='P{i}_{j}_0')")

    declare.append(f"P{layer}_{no}_0 = model.addVar(name='P{layer}_{no}_0')")

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                declare.append(f"X{i}_{j}_{time} = model.addVar(name='X{i}_{j}_{time}')")
                declare.append(f"P{i}_{j}_{time} = model.addVar(lb=-GRB.INFINITY, name='P{i}_{j}_{time}')")
                declare.append(f"S{i}_{j}_{time} = model.addVar(lb=-9999, name='S{i}_{j}_{time}')")
                declare.append(f"q{i}_{j}_{time} = model.addVar(vtype=gp.GRB.BINARY, name='q{i}_{j}_{time}')")
                declare.append(f"A{i}_{j}_{time} = model.addVar(vtype=gp.GRB.INTEGER, name='A{i}_{j}_{time}')")


    for time in range(1,time_steps+1):
        declare.append(f"S{layer}_{no}_{time} = model.addVar(lb=-9999, name='S{layer}_{no}_{time}')")
    
    # Encodings
    for i in range(1,layer):
        for j in range(1,len(w[i-1])+1):
            equations.append(f"model.addConstr(P{i}_{j}_0== 0)")
    equations.append(f"model.addConstr(P{layer}_{no}_0== 0)")

    thresh = 1
    lamb = 1
    M = 99999
    eps=0.0000001

    for time in range(1,time_steps+1):
        for i in range(1,layer):
            for j in range(1,len(w[i-1])+1):
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} + {M}* q{i}_{j}_{time} >= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(S{i}_{j}_{time} + P{i}_{j}_{time-1} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(X{i}_{j}_{time} >= 0)")
                equations.append(f"model.addConstr(X{i}_{j}_{time} <= {M}*(1-q{i}_{j}_{time}))")
                equations.append(f"model.addConstr(A{i}_{j}_{time} <= X{i}_{j}_{time})")
                equations.append(f"model.addConstr(A{i}_{j}_{time} + 1 >= X{i}_{j}_{time} + {eps})")
                equations.append(f"model.addConstr(P{i}_{j}_{time} == P{i}_{j}_{time-1} + S{i}_{j}_{time} - A{i}_{j}_{time})")
                equation = f'S{i}_{j}_{time} == ('
                if(i==1):
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_1)'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
                else:
                    for k in range(len(w[i-1][0])):
                        if(k!=0):
                            equation += f' + '
                        equation+=f'({w[i-1][j-1][k]} * A{i-1}_{k+1}_{time})'
                    equations.append(f"model.addConstr({equation}) + {b[i-1][j-1]})")
    
    out=f''
    for time in range(1,time_steps+1):
        if(time!=1):
            out+= '+'
        out+= f'S{layer}_{no}_{time}'
        
        
        equation = f'S{layer}_{no}_{time} == ('
        for k in range(len(w[layer-1][0])):
            if(k!=0):
                equation += f' + '
            equation+=f'(({w[layer-1][no-1][k]}) * A{layer-1}_{k+1}_{time})'
        equations.append(f"model.addConstr({equation})+ {b[layer-1][no-1]})")
    

    equations.append(f"model.addConstr({out}<={cond[0]*time_steps})")
    equations.append(f'model.setObjective({out}, gp.GRB.MINIMIZE)')


    return equations, declare

In [9]:
output_range = [-1.460299, -0.731794]

In [ ]:
delta = 0.0
output_range = [-1.460299 - delta , -0.731794 + delta]
print(output_range, delta)
for tym in range(1,7):
#     for rotate in range(1):
#     print('Round:',rotate)
    print('LB Check')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

    print('UB Check')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

[-1.460299, -0.731794] 0.0
LB Check
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20

Time steps 1
Property does not hold
Runtime 0.25799989700317383
UB Check

Time steps 1
Property does not hold
Runtime 0.08300018310546875
LB Check

Time steps 2
Property does not hold
Runtime 4.233000040054321
UB Check

Time steps 2
Property does not hold
Runtime 2.5930001735687256
LB Check

Time steps 3
Property does not hold
Runtime 9.754999876022339
UB Check

Time steps 3
Property does not hold
Runtime 4.983999967575073
LB Check

Time steps 4
Property does not hold
Runtime 37.115999937057495
UB Check

Time steps 4
Property does not hold
Runtime 7.799000024795532
LB Check

Time steps 5
Property does not hold
Runtime 57.08500003814697
UB Check

Time steps 5
Property does not hold
Runtime 25.46499991416931
LB Check


In [10]:
delta = 0.1
output_range = [-1.460299 - delta , -0.731794 + delta]
print(output_range, delta)
for tym in range(1,7):
    print('LB Check')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

    print('UB Check')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

[-1.560299, -0.631794] 0.1
LB Check
Set parameter Username
Academic license - for non-commercial use only - expires 2024-11-20

Time steps 1
3 Property holds
Runtime 1.1359999179840088
UB Check

Time steps 1
Property does not hold
Runtime 0.16000008583068848
LB Check

Time steps 2
3 Property holds
Runtime 5.233999967575073
UB Check

Time steps 2
Property does not hold
Runtime 2.4660000801086426
LB Check

Time steps 3
3 Property holds
Runtime 7.699000120162964
UB Check

Time steps 3
Property does not hold
Runtime 23.39299988746643
LB Check

Time steps 4
3 Property holds
Runtime 21.299000024795532
UB Check

Time steps 4
4 Property holds
Runtime 95.3769998550415
LB Check

Time steps 5
3 Property holds
Runtime 21.86300015449524
UB Check

Time steps 5
4 Property holds
Runtime 49.615999937057495
LB Check

Time steps 6
3 Property holds
Runtime 47.83299994468689
UB Check

Time steps 6
4 Property holds
Runtime 21.64900016784668


In [12]:
delta = 0.2
output_range = [-1.460299 - delta , -0.731794 + delta]
print(output_range, delta)
for tym in range(1,7):
    print('LB Check')
    equations, declare = can_go_below(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

    print('UB Check')
    equations, declare = can_go_above(weights,biases,layer_no,neuron_no,tym,output_range, input_bounds)
    model3 = summon_gurobi(declare, equations,0,3,0)
    print('\nTime steps',tym)
    if(model3.status in [2,10]):
        print('Property does not hold')
    elif(model3.status in [9]):
        print('Time out')
    else:
        print(model3.status, 'Property holds')
    print('Runtime',model3.Runtime)

[-1.660299, -0.5317939999999999] 0.2
LB Check

Time steps 1
3 Property holds
Runtime 0.20799994468688965
UB Check

Time steps 1
Property does not hold
Runtime 0.06399989128112793
LB Check

Time steps 2
3 Property holds
Runtime 1.246000051498413
UB Check

Time steps 2
4 Property holds
Runtime 2.880000114440918
LB Check

Time steps 3
3 Property holds
Runtime 3.2059998512268066
UB Check

Time steps 3
4 Property holds
Runtime 2.6389999389648438
LB Check

Time steps 4
3 Property holds
Runtime 12.1489999294281
UB Check

Time steps 4
4 Property holds
Runtime 7.9079999923706055
LB Check

Time steps 5
3 Property holds
Runtime 16.249000072479248
UB Check

Time steps 5
4 Property holds
Runtime 12.216000080108643
LB Check

Time steps 6
3 Property holds
Runtime 34.18400001525879
UB Check

Time steps 6
4 Property holds
Runtime 17.812999963760376
